In [1]:
import sys
sys.path.append('../')

In [2]:
# 📦 Imports
import pandas as pd
from sklearn.model_selection import train_test_split
from src.train import train_model, evaluate_model, save_model
from src.utils import load_data

In [3]:
df = load_data('../data/processed/processed_data.csv')
df.head()

Data loaded successfully from ../data/processed/processed_data.csv


,Amount,Value,Frequency,AvgAmount,AmountStdDev,Recency,ProductCategory_airtime,ProductCategory_data_bundles,ProductCategory_financial_services,ProductCategory_movies,...,ProductCategory_utility_bill,ChannelId_ChannelId_1,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_5,PricingStrategy_0,PricingStrategy_1,PricingStrategy_2,PricingStrategy_4,is_high_risk
0,0.643134,-0.031260,0.115536,-0.035832,-0.406095,-1.182989,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0
1,-0.809550,-1.832223,0.115536,-0.035832,-0.406095,-1.182989,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
2,0.542114,-0.353843,-2.147000,-0.189247,-5.996952,1.911180,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1
3,1.080223,1.404650,-0.573815,0.433114,0.799973,-0.142576,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0
4,-1.309389,-0.236095,-0.573815,0.433114,0.799973,-0.142576,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0


In [4]:
# 🎯 Prepare features and target
X = df.drop(columns=["is_high_risk"])
y = df["is_high_risk"]

In [5]:
# ✂️ Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=45)

In [6]:
from mlflow.tracking import MlflowClient

models = {
    "Logistic Regression": "logistic",
    "Random Forest": "random_forest",
    "Gradient Boosting": "xgboost"
}

results = []
returned_models = []

best_f1 = -1
best_model_info = {}

for display_name, internal_name in models.items():
    print(f"Training {display_name}...")

    model, run_id = train_model(X_train, y_train, model_name=internal_name)
    metrics = evaluate_model(model, X_test, y_test)

    f1 = metrics["f1_score"]

    results.append({
        "Model": display_name,
        "Accuracy": round(metrics["accuracy"], 3),
        "Precision": round(metrics["precision"], 3),
        "Recall": round(metrics["recall"], 3),
        "F1 Score": round(f1, 3),
        "ROC AUC": round(metrics["roc_auc"], 3)
    })

    returned_models.append(model)

    # Track the best model
    if f1 > best_f1:
        best_f1 = f1
        best_model_info = {
            "display_name": display_name,
            "internal_name": internal_name,
            "run_id": run_id
        }

Training Logistic Regression...


2025/07/01 17:19:26 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/01 17:19:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Training Random Forest...


2025/07/01 17:19:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/01 17:19:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Training Gradient Boosting...


c:\Users\user\Documents\Datasience\credict-risk-analysis\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:19:58] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
2025/07/01 17:19:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/01 17:20:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [8]:
import mlflow
# ✅ Register the best model
print(f"\nRegistering best model: {best_model_info['display_name']} (F1 Score: {round(best_f1, 3)})")

client = MlflowClient()
model_uri = f"runs:/{best_model_info['run_id']}/model"
registered_model_name = f"CreditRisk_{best_model_info['internal_name']}_model"

result = mlflow.register_model(model_uri, registered_model_name)

client.transition_model_version_stage(
    name=registered_model_name,
    version=result.version,
    stage="Production",
    archive_existing_versions=True
)

Successfully registered model 'CreditRisk_xgboost_model'.
2025/07/01 17:21:52 WARNING mlflow.tracking._model_registry.fluent: Run with id fc33e47344e44f7295136cdc5ed5077a has no artifacts at artifact path 'model', registering model based on models:/m-2ee0c611e3bc4e239638cb470f1ebeb8 instead



Registering best model: Gradient Boosting (F1 Score: 0.999)


Created version '1' of model 'CreditRisk_xgboost_model'.
C:\Users\user\AppData\Local\Temp\ipykernel_22464\3088280963.py:11: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1751379712536, current_stage='Production', deployment_job_state=None, description=None, last_updated_timestamp=1751379712636, metrics=[], model_id='m-2ee0c611e3bc4e239638cb470f1ebeb8', name='CreditRisk_xgboost_model', params={'max_depth': '3', 'n_estimators': '100'}, run_id='fc33e47344e44f7295136cdc5ed5077a', run_link=None, source='models:/m-2ee0c611e3bc4e239638cb470f1ebeb8', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [9]:
pd.DataFrame(results).set_index("Model")


,Accuracy,Precision,Recall,F1 Score,ROC AUC
Model,,,,,
Logistic Regression,0.994,0.991,0.964,0.977,0.999
Random Forest,0.999,1.000,0.995,0.998,1.000
Gradient Boosting,1.000,1.000,0.999,0.999,1.000


In [10]:
y.value_counts(normalize=True)

is_high_risk
0    0.87276
1    0.12724
Name: proportion, dtype: float64